In [ ]:
import pandas as pd
import seaborn as sns
import utils
import ETL

df = pd.read_csv('train.csv')
df = utils.encoding(df)

age_df = df[~df["Age"].isnull()]
age_df.head()

In [ ]:
corr = df.corr()
sns.heatmap(corr,  xticklabels=corr.columns,  yticklabels=corr.columns)

In [ ]:
corr["Age"].sort_values(ascending=False)

In [ ]:
from sklearn.model_selection import train_test_split

X = age_df[utils.Maps.age_predictors]
y = age_df['Age']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.33, shuffle=True, stratify=age_df["Pclass"])

In [ ]:
import xgboost as xgb
import numpy as np
import optuna
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from optuna.integration import XGBoostPruningCallback
from sklearn.model_selection import RepeatedKFold
from optuna import create_study

def objective(
    trial,
    X,
    y,
    random_state=22,
    n_splits=3,
    n_repeats=2,
    n_jobs=1,
    early_stopping_rounds=50,
):
    # XGBoost parameters
    params = {
        "verbosity": 0,  # 0 (silent) - 3 (debug)
        "objective": "reg:squarederror",
        "n_estimators": 10000,
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.05),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.6),
        "subsample": trial.suggest_loguniform("subsample", 0.4, 0.8),
        "alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "gamma": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 10, 1000),
        "seed": random_state,
        "n_jobs": n_jobs,
    }

    model = XGBRegressor(**params)
    pruning_callback = XGBoostPruningCallback(trial, "validation_0-rmse")
    rkf = RepeatedKFold(
        n_splits=n_splits, n_repeats=n_repeats, random_state=random_state
    )
    X_values = X.values
    y_values = y.values
    y_pred = np.zeros_like(y_values)
    for train_index, test_index in rkf.split(X_values):
        X_A, X_B = X_values[train_index, :], X_values[test_index, :]
        y_A, y_B = y_values[train_index], y_values[test_index]
        model.fit(
            X_A,
            y_A,
            eval_set=[(X_B, y_B)],
            eval_metric="rmse",
            verbose=0,
            callbacks=[pruning_callback],
            early_stopping_rounds=early_stopping_rounds,
        )
        y_pred[test_index] += model.predict(X_B)
    y_pred /= n_repeats
    return np.sqrt(mean_squared_error(y_train, y_pred))

In [ ]:
sampler = TPESampler(seed=124, multivariate=True)
study = create_study(direction="minimize", sampler=sampler)
study.optimize(
    lambda trial: objective(
        trial,
        X_train,
        y_train,
        random_state=124,
        n_splits=10,
        n_repeats=1,
        n_jobs=8,
        early_stopping_rounds=100,
    ),
    n_trials=2,
    n_jobs=1,
)

# display params
hp = study.best_params
for key, value in hp.items():
    print(f"{key:>20s} : {value}")
print(f"{'best objective value':>20s} : {study.best_value}")

In [ ]:
hp["verbosity"] = 0
hp["objective"] = "reg:squarederror"
hp["n_estimators"] = 10000
hp["seed"] = 124
hp["n_jobs"] = 8
model = XGBRegressor(**hp)
rkf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=124)
X_values = X_train.values
y_values = y_train.values
y_pred = np.zeros_like(y_test.values)
for train_index, test_index in rkf.split(X_values):
    X_A, X_B = X_values[train_index, :], X_values[test_index, :]
    y_A, y_B = y_values[train_index], y_values[test_index]
    model.fit(
        X_A,
        y_A,
        eval_set=[(X_B, y_B)],
        eval_metric="rmse",
        early_stopping_rounds=100,
        verbose=0,
    )
    y_pred += model.predict(X_test.values)
y_pred /= 1 * 10

In [ ]:
rmse = mean_squared_error(y_pred, y_test, squared=False)
rmse

In [ ]:
import pickle
with open('./age_model', 'wb') as files:
    pickle.dump(model, files)